In [ ]:
import numpy as np
import pandas as pd
import os

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
file_path = '/kaggle/input/stanford-earthquake-dataset-stead/merge.csv'

In [ ]:
!pip install basemap
!pip install basemap-data

In [ ]:
df = pd.read_csv(file_path, low_memory=False)
eqdf = df[df.source_magnitude > 0]

In [ ]:
eqdf.info()

In [ ]:
import matplotlib.pyplot as plt

longitude = eqdf['source_longitude']
latitude = eqdf['source_latitude']
magnitude = eqdf['source_magnitude']

plt.figure(figsize=(10, 6))
sc = plt.scatter(longitude, latitude, c=magnitude, cmap='viridis', alpha=0.5)
plt.colorbar(sc, label='Magnitude')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Heatmap of Earthquake Magnitudes')
plt.show()

In [ ]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371
    return c * r

def find_nearest_earthquake(eqdf, lat, lon):
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree')
    nbrs.fit(eqdf[['source_latitude', 'source_longitude']])

    input_point = np.array([[lat, lon]])

    distances, indices = nbrs.kneighbors(input_point)
    nearest_earthquake = eqdf.iloc[indices[0][0]]

    haversine_distance = haversine(lat, lon, nearest_earthquake['source_latitude'], nearest_earthquake['source_longitude'])
    
    return nearest_earthquake, haversine_distance

earthquake_data, distance = find_nearest_earthquake(eqdf, 45.00, 20.10)
print(f"Nearest earthquake details: \n{earthquake_data}")
print(f"Haversine distance to nearest earthquake: {distance} km")

# FREQUENCIES

In [ ]:
df = pd.read_csv(file_path, low_memory=False)
src_df = df[df.source_distance_km > 0]

In [ ]:
src_df.shape

In [ ]:
!pip install h5py tqdm

In [ ]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

ev_list = src_df['trace_name'].to_list()
file_name = "/kaggle/input/stanford-earthquake-dataset-stead/merge.hdf5"
dtfl = h5py.File(file_name, 'r')

for c, evi in enumerate(ev_list):
    dataset = dtfl.get('data/'+str(evi))
    if dataset:
        data = np.array(dataset)
        fft_data = np.fft.fft(data, axis=0)
        freq = np.fft.fftfreq(data.shape[0])

        positive_freq_indices = freq > 0
        positive_freqs = freq[positive_freq_indices]
        positive_fft_data = fft_data[positive_freq_indices]

        fig = plt.figure(figsize=(24, 18))

        all_peak_freqs = []  

        for i in range(3):  # E, N, Z channels
            ax = fig.add_subplot(3, 3, 3*i + 1)
            plt.plot(data[:, i], 'k')
            plt.title(f"Channel {i+1} Time Domain")
            plt.ylabel('Amplitude')
            plt.xlabel('Time')
            plt.tight_layout()

        max_freq = positive_freqs[-1]
        for i in range(3):
            ax = fig.add_subplot(3, 3, 3*i + 2)
            plt.plot(positive_freqs, np.abs(positive_fft_data[:, i]), 'r') 
            plt.xlim(0, max_freq)
            plt.title(f"Channel {i+1} Frequency Domain")
            plt.xlabel('Frequency (Hz)')
            plt.ylabel('Magnitude')
            plt.tight_layout()
            
        threshold = 0.8
        for i in range(3):
            ax = fig.add_subplot(3, 3, 3*i + 3)
            magnitude = np.abs(positive_fft_data[:, i])
            peaks, _ = find_peaks(magnitude, height=np.max(magnitude) * threshold)
            peak_freqs = positive_freqs[peaks]
            all_peak_freqs.append(peak_freqs)
            plt.vlines(peak_freqs, ymin=0, ymax=magnitude[peaks], color='b')
            plt.xlim(0, max_freq)
            plt.title(f"Channel {i+1} Peaks Only")
            plt.xlabel('Frequency (Hz)')
            plt.ylabel('Magnitude')
            plt.tight_layout()

        plt.show()

        for i, freqs in enumerate(all_peak_freqs):
            print(f"Peak frequencies for Channel {i+1}: {freqs}")

        print(f"Metadata for {evi}:")
        for key, value in dataset.attrs.items():
            print(f"{key}: {value}")

    break

In [ ]:
import numpy as np
import h5py
from scipy.signal import find_peaks
from tqdm import tqdm

file_name = "/kaggle/input/stanford-earthquake-dataset-stead/merge.hdf5"
dtfl = h5py.File(file_name, 'r')
features = []
labels = []

idx = 0

for evi in tqdm(src_df['trace_name'].to_list(), total=10000):
    idx += 1
    dataset = dtfl.get('data/'+str(evi))
    if dataset:
        source_distance = dataset.attrs.get('source_distance_km', None)
        source_magnitude = dataset.attrs.get('source_magnitude', None)
        source_depth = dataset.attrs.get('source_depth_km', None)

        if source_distance in [None, 'None'] or source_magnitude in [None, 'None'] or source_depth in [None, 'None']:
            continue

        complete_feature_set = [
            float(source_distance),
            float(source_magnitude),
            float(source_depth)
        ]

        data = np.array(dataset)
        fft_data = np.fft.fft(data, axis=0)
        freq = np.fft.fftfreq(data.shape[0])

        positive_freq_indices = freq > 0
        positive_freqs = freq[positive_freq_indices]
        positive_fft_data = fft_data[positive_freq_indices]

        weighted_frequencies = []

        for i in range(3):
            fft_magnitude = np.abs(positive_fft_data[:, i])
            peaks, properties = find_peaks(fft_magnitude, height=np.max(fft_magnitude) * 0.9)
            if len(peaks) > 0:
                peak_freqs = positive_freqs[peaks]
                peak_mags = properties['peak_heights']
                weighted_avg = np.average(peak_freqs, weights=peak_mags)
                weighted_frequencies.append(weighted_avg)
            else:
                weighted_frequencies.append(0)

        features.append(complete_feature_set)
        labels.append([round(freq, 3) for freq in weighted_frequencies])
    
    if idx > 9999:
        break

features = np.array(features, dtype=float)
labels = np.array(labels, dtype=float)

print("Features example:", features[:10])
print("Labels example:", labels[:10])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math

fig, axs = plt.subplots(3, 3, figsize=(15, 15))

lower_percentile = 10
upper_percentile = 99.5
intervals = 75

for i in range(3):
    for j in range(3):
        axs[i, j].scatter(features[:, j], labels[:, i], label=f'Label {i}', alpha=0.5, color='grey')

        min_value = np.min(features[:, j])
        max_value = np.max(features[:, j])
        range_width = (max_value - min_value) / intervals

        for section in range(intervals):
            section_min = min_value + section * range_width
            section_max = section_min + range_width
            
            section_indices = (features[:, j] >= section_min) & (features[:, j] < section_max)
            section_features = features[section_indices, j]
            section_labels = labels[section_indices, i]

            if len(section_labels) > 0:
                low_limit = np.percentile(section_labels, lower_percentile)
                high_limit = np.percentile(section_labels, upper_percentile)
                clean_indices = (section_labels > low_limit) & (section_labels < high_limit)
                clean_features = section_features[clean_indices]
                clean_labels = section_labels[clean_indices]

                axs[i, j].scatter(clean_features, clean_labels, color='blue', alpha=0.5)

                if len(clean_labels) > 0:
                    top_indices = np.argsort(clean_labels)[-1:]
                    top_features = clean_features[top_indices]
                    top_labels = clean_labels[top_indices]

                    axs[i, j].scatter(top_features, top_labels, color='red', s=50)

        axs[i, j].set_xlabel(f'Feature {j} Value')
        axs[i, j].set_ylabel(f'Label {i} Value')
        axs[i, j].legend()
        axs[i, j].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import tensorflow as tf
from tensorflow.keras import layers, models

def exponential_decreasing(x, a, b):
    return a * np.exp(-b * x)

def build_model():
    model = models.Sequential([
        layers.Dense(64, activation='relu', input_shape=(1,)),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    return model

num_top_values = 3
start_value = 20
end_value = np.max(features[:, 0])

fig, axs = plt.subplots(1, 3, figsize=(30, 10))
models = [build_model() for _ in range(3)]  # Create a separate model for each label

for label_index in range(3):
    top_features = []
    top_labels = []

    valid_indices = features[:, 0] >= start_value
    valid_features = features[valid_indices]
    valid_labels = labels[valid_indices]

    feature_index = 0

    axs[label_index].scatter(valid_features[:, feature_index], valid_labels[:, label_index], label=f'Label {label_index}', alpha=0.5, color='grey')

    min_value = np.min(valid_features[:, feature_index])
    max_value = np.max(valid_features[:, feature_index])
    range_width = (max_value - min_value) / intervals

    for section in range(intervals):
        section_min = min_value + section * range_width
        section_max = section_min + range_width
        
        section_indices = (valid_features[:, feature_index] >= section_min) & (valid_features[:, feature_index] < section_max)
        section_features = valid_features[section_indices, feature_index]
        section_labels = valid_labels[section_indices, label_index]

        if len(section_labels) > 0:
            low_limit = np.percentile(section_labels, lower_percentile)
            high_limit = np.percentile(section_labels, upper_percentile)
            clean_indices = (section_labels > low_limit) & (section_labels < high_limit)
            clean_features = section_features[clean_indices]
            clean_labels = section_labels[clean_indices]

            axs[label_index].scatter(clean_features, clean_labels, color='blue', alpha=0.5)

            if len(clean_labels) > 0:
                top_indices = np.argsort(clean_labels)[-num_top_values:]
                top_features.extend(clean_features[top_indices])
                top_labels.extend(clean_labels[top_indices])

    axs[label_index].scatter(top_features, top_labels, color='red', s=50)

    axs[label_index].set_xlabel(f'Feature {feature_index} Value')
    axs[label_index].set_ylabel(f'Label {label_index} Value')
    axs[label_index].legend()
    axs[label_index].grid(True)

    if len(top_features) > 0:
        top_features = np.array(top_features)
        top_labels = np.array(top_labels)

        log_labels = np.log(top_labels)
        
        models[label_index].fit(top_features[:, np.newaxis], log_labels, epochs=150, verbose=0)
        
        x_fit = np.linspace(start_value, end_value, 100)
        log_y_fit = models[label_index].predict(x_fit[:, np.newaxis]).flatten()
        y_fit = np.exp(log_y_fit)
        
        axs[label_index].plot(x_fit, y_fit, color='green', label='NN Exponential Fit for Top Values')

plt.tight_layout()
plt.show()

def predict_x(input_feature):
    log_y = models[0].predict(np.array([[input_feature]]))
    return np.exp(log_y.flatten())

def predict_y(input_feature):
    log_y = models[1].predict(np.array([[input_feature]]))
    return np.exp(log_y.flatten())

def predict_z(input_feature):
    log_y = models[2].predict(np.array([[input_feature]]))
    return np.exp(log_y.flatten())

In [ ]:
print(predict_x(250))
print(predict_y(25))
print(predict_z(25))

In [ ]:
in_lat = 45.00
in_lon = 20.10

earthquake_data, distance_km = find_nearest_earthquake(eqdf, in_lat, in_lon)
x_freq = predict_x(distance_km)
y_freq = predict_y(distance_km)
z_freq = predict_z(distance_km)

In [ ]:
print(x_freq, y_freq, z_freq)